In [1]:
import sys
!{sys.executable} "-m pip install pandas requests tqdm"


'c:\Users\WINDOWS' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install pandas requests tqdm



Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [3]:
!{sys.executable} -m pip install pandas requests tqdm

'c:\Users\WINDOWS' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import pandas as pd
import requests
import time
from tqdm import tqdm

# ==============================
# CONFIGURATION
# ==============================

SMARTY_AUTH_ID = "54a4c77e-8649-1ccf-d4cf-cbd2fcbe0688"
SMARTY_AUTH_TOKEN = "mOVuWjU3mi3Gje7ft9yn"

INPUT_FILE = "addresses.csv"
OUTPUT_FILE = "addresses_standardized.csv"
FAILED_FILE = "failed_addresses.csv"
REPORT_FILE = "summary_report.txt"

MAX_RETRIES = 3
RETRY_DELAY = 2  # seconds

# ==============================
# FUNCTIONS
# ==============================

def call_smarty_api(street, city, state, zipcode):
    """Call the Smarty US Street API for a given address with retry handling."""
    if not street or not state:
        return None

    url = "https://us-street.api.smarty.com/street-address"
    params = {
        "auth-id": SMARTY_AUTH_ID,
        "auth-token": SMARTY_AUTH_TOKEN,
        "street": street,
        "city": city or "",
        "state": state or "",
        "zipcode": zipcode or "",
        "candidates": 1,
    }

    for attempt in range(MAX_RETRIES):
        try:
            response = requests.get(url, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if len(data) > 0:
                    result = data[0]
                    metadata = result.get("metadata", {})
                    components = result.get("components", {})

                    return {
                        "StandardizedAddress": result.get("delivery_line_1", ""),
                        "LastLine": result.get("last_line", ""),
                        "DPVStatus": metadata.get("dpv_match_code", ""),
                        "Latitude": metadata.get("latitude", ""),
                        "Longitude": metadata.get("longitude", ""),
                        "ZIPCode": components.get("zipcode", ""),
                        "ZIPPlus4": components.get("plus4_code", ""),
                    }
            else:
                print(f"⚠️ Error {response.status_code} for {street}")
        except Exception as e:
            print(f"❌ Error ({attempt+1}/{MAX_RETRIES}) for {street}: {e}")

        time.sleep(RETRY_DELAY)
    return None


def classify_address_issue(row):
    if row["DPVStatus"] == "N":
        return "Invalid or Undeliverable"
    elif pd.isna(row["StandardizedAddress"]) or row["StandardizedAddress"] == "":
        return "Could not standardize"
    elif row["IsDuplicate"]:
        return "Duplicate"
    else:
        return "Clean"


# ==============================
# MAIN SCRIPT
# ==============================

def main():
    print("📥 Loading input file...")
    df = pd.read_csv(INPUT_FILE)

    # Combine house number + street name into one field
    df["StreetFull"] = df[["Number", "Street"]].astype(str).agg(" ".join, axis=1).str.strip()

    results = []
    failed_rows = []

    print("🔍 Standardizing addresses via Smarty API...")

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        street = row.get("StreetFull")
        city = row.get("Suburb")
        state = row.get("State")
        zipcode = row.get("PostalCode")

        api_result = call_smarty_api(street, city, state, zipcode)

        if api_result:
            results.append(api_result)
        else:
            results.append({
                "StandardizedAddress": "",
                "LastLine": "",
                "DPVStatus": "",
                "Latitude": "",
                "Longitude": "",
                "ZIPCode": "",
                "ZIPPlus4": "",
            })
            failed_rows.append(row)

    # Merge results back into main dataframe
    results_df = pd.DataFrame(results)
    df = pd.concat([df.reset_index(drop=True), results_df], axis=1)

    # Identify duplicates
    df["IsDuplicate"] = df.duplicated(subset=["StandardizedAddress"], keep=False)
    df["AddressStatus"] = df.apply(classify_address_issue, axis=1)

    # Generate summary report
    print("\n📊 Address Quality Summary:")
    summary = df["AddressStatus"].value_counts()
    print(summary)

    with open(REPORT_FILE, "w") as f:
        f.write("📋 Address Standardization Summary Report\n")
        f.write("=====================================\n\n")
        f.write(summary.to_string())
        f.write("\n\nTotal rows processed: " + str(len(df)))
        f.write("\nFailed API lookups: " + str(len(failed_rows)))

    # Save results
    df.to_csv(OUTPUT_FILE, index=False)

    if failed_rows:
        failed_df = pd.DataFrame(failed_rows)
        failed_df.to_csv(FAILED_FILE, index=False)
        print(f"⚠️ Some addresses failed. Logged to {FAILED_FILE}")

    print(f"\n✅ All done! Clean data saved to {OUTPUT_FILE}")
    print(f"📑 Summary report saved to {REPORT_FILE}")


if __name__ == "__main__":
    main()


📥 Loading input file...
🔍 Standardizing addresses via Smarty API...


  6%|▌         | 823/14050 [16:30<2:13:50,  1.65it/s] 

⚠️ Error 402 for 9701 S 11th Ave
⚠️ Error 402 for 9701 S 11th Ave
⚠️ Error 402 for 9701 S 11th Ave


  6%|▌         | 824/14050 [16:38<10:05:15,  2.75s/it]

⚠️ Error 402 for 9706 S 11th Ave
⚠️ Error 402 for 9706 S 11th Ave
⚠️ Error 402 for 9706 S 11th Ave


...

⚠️ Error 402 for 3500 W Manchester Blvd
⚠️ Error 402 for 3500 W Manchester Blvd
⚠️ Error 402 for 3500 W Manchester Blvd


  8%|▊         | 1169/14050 [1:02:18<28:56:37,  8.09s/it]

⚠️ Error 402 for 3500 W Manchester Blvd
⚠️ Error 402 for 3500 W Manchester Blvd
⚠️ Error 402 for 3500 W Manchester Blvd


  8%|▊         | 1170/14050 [1:02:26<28:57:33,  8.09s/it]

⚠️ Error 402 for 3500 W Manchester Blvd
⚠️ Error 402 for 3500 W Manchester Blvd
⚠️ Error 402 for 3500 W Manchester Blvd


In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

# --- CONFIGURATION ---
AUTH_ID = "54a4c77e-8649-1ccf-d4cf-cbd2fcbe0688"
AUTH_TOKEN = "mOVuWjU3mi3Gje7ft9yn"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

# --- LOAD INPUT ---
df = pd.read_csv("sample.csv")

# --- NEW OUTPUT COLUMNS ---
df["StandardizedAddress"] = ""
df["DPVStatus"] = ""
df["Latitude"] = ""
df["Longitude"] = ""
df["AddressStatus"] = ""

# --- PROCESS EACH ADDRESS ---
for i, row in tqdm(df.iterrows(), total=len(df)):
    street = f"{row['Number']} {row['Street']}"
    city = row.get("Suburb", "")
    state = row.get("State", "CA")
    zipcode = str(row.get("PostalCode", ""))

    params = {
        "auth-id": AUTH_ID,
        "auth-token": AUTH_TOKEN,
        "street": street,
        "city": city,
        "state": state,
        "zipcode": zipcode,
    }

    try:
        response = requests.get(SMARTY_URL, params=params, timeout=10)

        # Error 402 = no more lookups
        if response.status_code == 402:
            print(f"⚠️ Out of lookups for {street}. Check your API balance.")
            df.loc[i, "AddressStatus"] = "API limit reached"
            continue
        elif response.status_code != 200:
            print(f"⚠️ Error {response.status_code} for {street}")
            df.loc[i, "AddressStatus"] = f"Error {response.status_code}"
            continue

        data = response.json()
        if not data:
            df.loc[i, "AddressStatus"] = "No match"
            continue

        # Take the first matched result
        result = data[0]
        components = result.get("components", {})
        metadata = result.get("metadata", {})
        analysis = result.get("analysis", {})

        standardized = result.get("delivery_line_1", "")
        if "delivery_line_2" in result:
            standardized += " " + result["delivery_line_2"]
        standardized += ", " + result.get("last_line", "")

        # Fill in standardized data
        df.loc[i, "StandardizedAddress"] = standardized
        df.loc[i, "DPVStatus"] = analysis.get("dpv_match_code", "")
        df.loc[i, "Latitude"] = metadata.get("latitude", "")
        df.loc[i, "Longitude"] = metadata.get("longitude", "")
        df.loc[i, "AddressStatus"] = "Validated"

    except Exception as e:
        print(f"⚠️ Exception for {street}: {e}")
        df.loc[i, "AddressStatus"] = "Error"

# --- SAVE OUTPUT ---
df.to_csv("standardized_output.csv", index=False)
print("✅ Processing complete! Results saved to standardized_output.csv")
